In [35]:
import pymongo
import pandas as pd
import jieba
import numpy as np
import string
from collections import Counter

In [16]:
# mongodb uri
conn_str = "mongodb://1.15.118.125:27017/"
# 客户端连接
client = pymongo.MongoClient(conn_str, serverSelectionTimeoutMS=5000)
collection_user = client.get_database("Lab").get_collection("Mission")

In [59]:
# 查询 title 与对应的 tags
title_with_tags = {}
for doc in collection_user.find():
    mission_tags = doc.get("missionTags")
    mission_title = doc.get("title")
    title_with_tags[mission_title] = mission_tags

# pands 格式化输出
ser_tag_tags = pd.Series(title_with_tags)
ser_tag_tags.head(len(ser_tag_tags))

选调知多少政策宣讲活动                                                                                                           [就业发展, 宣讲]
会计专业课程组本科教学汇报                                                                                                      [教学建设, 报告/汇报]
喻园管理论坛: Understanding the Digital Resilience of Physicians during the COVID-19 Pandemic: An Empirical Study             [学术, 讲座]
国奖专访                                                                                                                  [采访, 校园生活]
AOL第三轮工作会议                                                                                                            [教学建设, 会议]
校运会跟拍                                                                                                                 [文体活动, 赛事]
学习贯彻党的二十大精神宣讲报告会                                                                                               [党团活动, 宣讲, 报告/汇报]
“廖建桥教室”揭牌仪式活动                                                                                    

In [60]:
# 忽略词
eng_words = list(string.ascii_lowercase + string.ascii_uppercase)
exclude_words = ['，', '“', '”', '、', '《', '》', '。', ':', '-', '的', '?', ' ', ',', '&']
for w in eng_words:
    exclude_words.append(w)
for i in range(10):
    exclude_words.append(str(i))
print(exclude_words)

['，', '“', '”', '、', '《', '》', '。', ':', '-', '的', '?', ' ', ',', '&', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [61]:
# 分割 title 的词语
total_words = []
title_with_words = {}
for title in title_with_tags.keys():
    for i in exclude_words:
        title = title.replace(i, '')
    title_with_words[title] = []
    # jieba拆分方法
    words = jieba.cut(title, cut_all=False)
    for word in words:
        total_words.append(word)
        title_with_words[title].append(word)

fre = Counter(total_words)
word_fre = sorted(fre.items(), key=lambda i: i[1], reverse=True)
print("总词数: " + len(word_fre).__str__())
print(word_fre)

总词数: 79
[('管理', 7), ('论坛', 7), ('活动', 5), ('喻园', 5), ('宣讲', 3), ('二十大', 2), ('报告会', 2), ('仪式', 2), ('舍导', 2), ('交流', 2), ('学院', 2), ('第二届', 2), ('工商管理学', 2), ('科', 2), ('高端', 2), ('系列', 2), ('选调', 1), ('知多少', 1), ('政策', 1), ('会计专业', 1), ('课程', 1), ('组', 1), ('本科', 1), ('教学', 1), ('汇报', 1), ('国奖', 1), ('专访', 1), ('第三轮', 1), ('工作', 1), ('会议', 1), ('校运会', 1), ('跟', 1), ('拍', 1), ('学习', 1), ('贯彻', 1), ('党', 1), ('精神', 1), ('廖', 1), ('建桥', 1), ('教室', 1), ('揭牌', 1), ('领学团', 1), ('第一次', 1), ('集体', 1), ('备课', 1), ('会', 1), ('华工', 1), ('杯', 1), ('乒乓球赛', 1), ('半决赛', 1), ('热缩', 1), ('片', 1), ('发展', 1), ('对象', 1), ('开班', 1), ('杨院长', 1), ('教授', 1), ('党课', 1), ('十一月', 1), ('学术', 1), ('学科建设', 1), ('座谈会', 1), ('新进', 1), ('老师', 1), ('培训', 1), ('学生', 1), ('骨干', 1), ('训练营', 1), ('结营', 1), ('十二月', 1), ('中国', 1), ('管理学', 1), ('前沿', 1), ('研究', 1), ('研讨会', 1), ('开幕式', 1), ('与', 1), ('主', 1), ('报告', 1)]


In [62]:
print(title_with_words)

{'选调知多少政策宣讲活动': ['选调', '知多少', '政策', '宣讲', '活动'], '会计专业课程组本科教学汇报': ['会计专业', '课程', '组', '本科', '教学', '汇报'], '喻园管理论坛': ['喻园', '管理', '论坛'], '国奖专访': ['国奖', '专访'], '第三轮工作会议': ['第三轮', '工作', '会议'], '校运会跟拍': ['校运会', '跟', '拍'], '学习贯彻党二十大精神宣讲报告会': ['学习', '贯彻', '党', '二十大', '精神', '宣讲', '报告会'], '廖建桥教室揭牌仪式活动': ['廖', '建桥', '教室', '揭牌', '仪式', '活动'], '领学团第一次集体备课会': ['领学团', '第一次', '集体', '备课', '会'], '华工杯乒乓球赛半决赛': ['华工', '杯', '乒乓球赛', '半决赛'], '热缩片活动': ['热缩', '片', '活动'], '发展对象开班杨院长教授党课': ['发展', '对象', '开班', '杨院长', '教授', '党课'], '十一月舍导交流': ['十一月', '舍导', '交流'], '管理学院第二届工商管理学科高端论坛系列活动学术报告会': ['管理', '学院', '第二届', '工商管理学', '科', '高端', '论坛', '系列', '活动', '学术', '报告会'], '管理学院第二届工商管理学科高端论坛系列活动学科建设座谈会': ['管理', '学院', '第二届', '工商管理学', '科', '高端', '论坛', '系列', '活动', '学科建设', '座谈会'], '二十大宣讲': ['二十大', '宣讲'], '新进老师培训': ['新进', '老师', '培训'], '学生骨干训练营结营仪式': ['学生', '骨干', '训练营', '结营', '仪式'], '十二月舍导交流': ['十二月', '舍导', '交流'], '中国管理学前沿研究研讨会开幕式与主报告': ['中国', '管理学', '前沿', '研究', '研讨会', '开幕式', '与', '主', '报告']}


In [63]:
# 查询 tag 的频次
def get_tag_times(dict_value_tags):
    tag_with_time = {}
    for key in dict_value_tags:
        for tag in dict_value_tags[key]:
            if tag not in tag_with_time:
                tag_with_time[tag] = 1
            else:
                tag_with_time[tag] += 1
    return tag_with_time


# 获取 tags 列表
tags_list = list(get_tag_times(title_with_tags).keys())
print(len(tags_list))

16


In [64]:
dist = np.zeros((len(title_with_tags), len(set(total_words))))
print(dist.shape)

(24, 79)


In [ ]:
import tensorflow as tf

num_outputs = len(tags_list)
tf.random.set_seed(1)
item_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_outputs),
])